In [1]:
#!/usr/bin/env python3
from collections import Counter
import time, sys
#from csv_combinator import Combinator
#from data_extraction import *
#from pepper_arcs.arcs import *
#from data_extraction.clean_csv import cleancsv, WrapperGetPart

%run 'Code csv combinator.ipynb'
%run 'Code data_extraction.ipynb'
%run 'Code arcs.ipynb'

import pandas as pd

REGEX = r'(?<=P)[0-9]*'
PNUM_COLUMN_NAME = 'pNum'
ENUM_COLUMN_NAME = 'eNum'

NEW_RECORDINGS_FOLDER = r'new_recording/'
NEW_RECORDINGS_COMBINED = r'new_data/combined.csv'
CLEANED_ROTATED_DATA = r'new_data/cleaned_rotated_combined.csv'
SIDED_DATA = r'new_data/sided_data.csv'
ARCS = r'new_data/combined_arcs.csv'

INPUT_FOLDER = r'new_recording/'
OUTPUT_FOLDER = r"new_data/"

COMBINED_DATA_FILE = r'combined.csv'
CLEANED_DATA_FILE = r'cleaned.csv'
CUT_SIDED_DATA_FILE = r'cut_sided.csv'
ROTATED_DATA_FILE = r'rotated.csv'
ARCS_CALCULATED_FILE = r'arcs_calculated.csv'

def remove_multiple_bodies(path):
    df = pd.read_csv(path)
    per_pnum = [x for _, x in df.groupby(df.pNum)]
    df_list = []

    for pnum in per_pnum:
        per_enum = [x for _, x in pnum.groupby(pnum.eNum)]
        for enum in per_enum:
            trackings = [x for _, x in enum.groupby(enum.trackingId)]
            df_list.append(max(trackings, key=len))

    return pd.concat(df_list)
    
def run_old():
    c = Combinator(NEW_RECORDINGS_FOLDER, NEW_RECORDINGS_COMBINED)
    c.combine(REGEX, PNUM_COLUMN_NAME, ENUM_COLUMN_NAME)
    print("remove bodies")
    removed_bodies = remove_multiple_bodies(NEW_RECORDINGS_COMBINED)
    
    print("rotate bodies")
    per_pnum1 = [x for _, x in removed_bodies.groupby(removed_bodies.pNum)]
    rotated = None
    
    for person in per_pnum1:
        print(person.iloc[0].pNum)
        per_enum = [x for _, x in person.groupby(person['eNum'])]
        for exercise in per_enum:
            print(exercise.iloc[0].eNum)
            rotated_e = rotate_body(exercise)
            if type(rotated) == type(None):
                rotated = rotated_e
            else:
                rotated= rotated.append(rotated_e)
    
    try:
        old_rotated = pd.read_csv(CLEANED_ROTATED_DATA)        
        new_rotated = old_rotated.append(rotated)
    except:
        new_rotated = rotated
    new_rotated.to_csv(CLEANED_ROTATED_DATA)


    print("cut bodies")
    # Do Thijs' stuff
    per_pnum2 = [x for _, x in rotated.groupby(rotated.pNum)]
    combined_with_sides = None
    for person in per_pnum2:
        per_enum = [x for _, x in person.groupby(person['eNum'])]
        if len(per_enum) == 3:
            for exercise in per_enum:
                try:
                    enum_ = exercise.iloc[0].eNum
                    exercise = exercise.reset_index(drop=True)
                    cut_exercise = cleancsv(exercise, enum_)
                    cut_exercise = cut_exercise.reset_index(drop=True)
                    cut_exercise = WrapperGetPart(cut_exercise, enum_)                    
                    if type(combined_with_sides) == type(None):
                        combined_with_sides = cut_exercise
                    else:
                        combined_with_sides = pd.append(cut_exercise)
                except:
                    print("EXCEPTION!")
                    continue

    try:
        old_sided = pd.read_csv(SIDED_DATA)
        new_sided = old_sided.append(combined_with_sides)
    except:
        new_sided = combined_with_sides        
    new_sided.to_csv(SIDED_DATA)
    
    print("calculate bodies")
    # Calculate arcs
    per_pnum3 = [x for _, x in combined_with_sides.groupby(combined_with_sides.pNum)]
    all_arcs = None
    for person in per_pnum3:
        pnum = person['pNum'].iloc[0]
        print(pnum)
        per_enum = [x for _, x in person.groupby(person.eNum)]
        for exercise in per_enum:
            enum_ = exercise['eNum'].iloc[0]
            per_side = [x for _, x in exercise.groupby(exercise.Side)]
            for sided in per_side:
                Side = sided['Side'].iloc[0]
                arcs = get_arcs(sided, enum_, Side)
                if not Side == '':
                    arcs = get_arcs(sided, enum_, Side)
                    arcs['pNum'] = pnum
                    arcs['eNum'] = enum_
                    arcs['Side'] = Side
                    if type(all_arcs) == type(None):
                        all_arcs = arcs
                        print("all_arcs is None")
                    else:
                        all_arcs = all_arcs.append(arcs)
                        print("APPENDED")

    try:
        old_arcs = pd.read_csv(ARCS)
        new_arcs = old_arcs.append(all_arcs)
    except:
        new_arcs = all_arcs

    new_arcs.to_csv(r'/data/pepper/upto_92.csv')

    print("jwz")
    
def run():
    print("should run")
    
def try_read_csv(fileName, decimal='.'):
    try:
        df = pd.read_csv(fileName, index_col=0, decimal=decimal)
    except:
        df = pd.DataFrame()
    return df

def append_avoiding_duplicating(current_data, new_data):
    if not current_data.empty:
        persons_current = current_data.pNum.unique()
        valid_new_data = new_data[~new_data['pNum'].isin(persons_current)]
        data = current_data.append(valid_new_data)
    else:
        data = new_data
    return data

def get_dataframe_delta(current_data, containing_new_data):
    if not current_data.empty and not containing_new_data.empty:
        currentPersons = current_data.pNum.unique()
        dataNotInCurrent = containing_new_data[~containing_new_data.pNum.isin(currentPersons)]
        return dataNotInCurrent
    else:
        return containing_new_data

def combine_data(input_folder, current_combined):
    if len(current_combined) > 0:
        currentPersons = current_combined.pNum.unique()
    else:
        currentPersons = []
            
    new_combined = pd.DataFrame()
    if not os.path.exists(input_folder):
        raise FileNotFoundError()
    for root, dirs, files in os.walk(input_folder, topdown=True):
        dirs.sort()
        files.sort()
        fileCounter = 1        
        regResult = re.search(REGEX, root)        
        if type(regResult) != type(None):
            personNumber = int(regResult.group(0))
            if not personNumber in currentPersons:
                #print("Adding Person {0}".format(personNumber))                
                for file in files:
                    fullpath = root + '/' + file
                    if fullpath[-3:] == 'csv':           
                        new_data = pd.read_csv(fullpath)
                        new_data[PNUM_COLUMN_NAME] = personNumber;
                        new_data[ENUM_COLUMN_NAME] = fileCounter;
                        new_combined = new_combined.append(new_data)
                        fileCounter+=1
                        sys.stdout.write("Added Person {0} exercise {1}".format(personNumber, fileCounter) + '\r')    
    if not new_combined.empty:
        combined_data = current_combined.append(new_combined)
        print("Combining data finished, persons added: {0}".format(new_combined.pNum.unique()))
    else:
        combined_data = current_combined
        print("Combining data finished, no new persons detected")
    return combined_data, new_combined
      
def clean_data(dataToClean):
    per_pnum = [x for _, x in dataToClean.groupby(dataToClean.pNum)]
    df_list = []

    for pnum in per_pnum:
        per_enum = [x for _, x in pnum.groupby(pnum.eNum)]
        for enum in per_enum:
            trackings = [x for _, x in enum.groupby(enum.trackingId)]
            df_list.append(max(trackings, key=len))
    single_bodies = pd.concat(df_list)        
    
    print("Cleaning finished, persons cleaned: {0}".format(single_bodies.pNum.unique()))
    return single_bodies
    
def cut_frames_add_moving_sides(dataToCut):
    # Do Thijs' stuff
    per_pnum2 = [x for _, x in dataToCut.groupby(dataToCut.pNum)]
    cut_with_sides = pd.DataFrame()
    for person in per_pnum2:
        per_enum = [x for _, x in person.groupby(person['eNum'])]
        if len(per_enum) == 3:
            for exercise in per_enum:
                try:
                    sys.stdout.write("Cutting Person {0} exercise {1}  ".format(person.iloc[0].pNum, exercise.iloc[0].eNum) + '\r')
                    enum_ = exercise.iloc[0].eNum
                    exercise = exercise.reset_index(drop=True)
                    cut_exercise = cleancsv(exercise, enum_)
                    cut_exercise = cut_exercise.reset_index(drop=True)
                    cut_exercise = WrapperGetPart(cut_exercise, enum_)                    

                    cut_with_sides = cut_with_sides.append(cut_exercise)
                except:
                    print("EXCEPTION:  Person {0} exercise {1}".format(person.iloc[0].pNum, exercise.iloc[0].eNum))
                    continue     
    if not cut_with_sides.empty:
        print("Cutting frames finished, person cut andf sided: {0}".format(cut_with_sides.pNum.unique()))
    else:
        print("Cutting frames finished, no new persons detected")
    return cut_with_sides    

def rotate_data(dataToRotate):
    per_pnum1 = [x for _, x in dataToRotate.groupby(dataToRotate.pNum)]
    rotated = pd.DataFrame()

    for person in per_pnum1:      
        per_enum = [x for _, x in person.groupby(person['eNum'])]
        for exercise in per_enum:
            sys.stdout.write("Rotating Person {0} exercise {1}  ".format(person.iloc[0].pNum, exercise.iloc[0].eNum) + '\r')
            rotated_e = rotate_body(exercise)
            rotated = rotated.append(rotated_e)      
    if not rotated.empty:
        print("Rotating data finished, person rotated: {0}".format(rotated.pNum.unique()))
    else:
        print("Rotating data finished, no new persons detected")
    return rotated
    
def calculate_angles(dataToProcess):
    # Calculate arcs
    per_pnum3 = [x for _, x in dataToProcess.groupby(dataToProcess.pNum)]
    all_arcs = pd.DataFrame()
    for person in per_pnum3:
        pnum = person['pNum'].iloc[0]
        per_enum = [x for _, x in person.groupby(person.eNum)]
        for exercise in per_enum:
            sys.stdout.write("Calculating angle for Person {0} exercise {1}  ".format(person['pNum'].iloc[0], exercise['eNum'].iloc[0]) + '\r')
            enum_ = exercise['eNum'].iloc[0]
            per_side = [x for _, x in exercise.groupby(exercise.Side)]
            for sided in per_side:
                Side = sided['Side'].iloc[0]
                #arcs = get_arcs(sided, pnum, enum_, Side)
                if not Side == '':
                    arcs = get_arcs(sided, pnum, enum_, Side)
                    arcs['pNum'] = pnum
                    arcs['eNum'] = enum_
                    arcs['Side'] = Side
                    all_arcs = all_arcs.append(arcs)
    print("calculating angles finished")
    return all_arcs
    

